In [ ]:
!pip freeze
!pip3 install numpy
!pip3 install pandas
!pip3 install sklearn
!pip3 install matplotlib
!pip3 install pydotplus
!pip3 install six

Problem 1: The American Job

1. Bad data

In [68]:
import pandas as pd

df = pd.read_csv('https://gist.githubusercontent.com/inumanag/713fb4367cacfeab82b07104482e73a7/raw/76a7ab7503476ce8f6a25269dc2755c13bbf5c36/elections.csv')
df = df.apply(pd.to_numeric, errors='coerce')

nan_val = df.isna().values.sum()
print('Not a number values: ',nan_val)

df = df.fillna(0)

Not a number values:  5


2. Split

In [69]:
# encoding tip from: https://stackoverflow.com/questions/61264795/pandas-unicodedecodeerror-utf-8-codec-cant-decode-bytes-in-position-0-1-in
# was getting unicode error otherwise
df2 = pd.read_csv('https://raw.githubusercontent.com/kjhealy/us-county/master/data/census/fips-by-state.csv', encoding = 'unicode_escape', engine = 'python')

df2.rename(columns = {'fips':'FIPS', 'state':'STATE', 'county':'COUNTY'}, inplace = True)
df = pd.merge(df, df2, on="FIPS", how="inner")
del df['FIPS']

3. Aggregate

Aggregate Education

In [80]:
ed_col = ['PERCENT OF ADULTS WITH LESS THAN A HIGH SCHOOL DIPLOMA,', 
                     'PERCENT OF ADULTS WITH A HIGH SCHOOL DIPLOMA ONLY,',
                     "PERCENT OF ADULTS COMPLETING SOME COLLEGE OR ASSOCIATE'S DEGREE,",
                     "PERCENT OF ADULTS WITH A BACHELOR'S DEGREE OR HIGHER,"]

def aggEd(row):
  col_val_dict = {'LT High School': row[ed_col[0]], 
                'High School': row[ed_col[1]], 
                'Some College': row[ed_col[2]],
                "GTE Bachelor's": row[ed_col[3]]}
  # help from https://stackoverflow.com/questions/65328275/find-key-in-dictionary-with-n-th-largest-value
  sort = sorted(col_val_dict.items(), key = lambda item: item[1], reverse = True)
  highest = sort[2][0]
  return highest

# help from https://towardsdatascience.com/create-new-column-based-on-other-columns-pandas-5586d87de73d
df['EDUCATION'] = df.apply(lambda row: aggEd(row), axis=1)

Aggregate Religion

In [87]:
rel_col = ['AMISH', 'BUDDHIST', 'CATHOLIC',
       'CHRISTIAN GENERIC', 'EASTERN ORTHODOX', 'HINDU', 'JEWISH',
       'MAINLINE CHRISTIAN', 'MORMON', 'MUSLIM', 'NON-CATHOLIC CHRISTIAN',
       'OTHER', 'OTHER CHRISTIAN', 'OTHER MISC', 'PENTECOSTAL / CHARISMATIC',
       'PROTESTANT DENOMINATION', 'ZOROASTRIAN',]

rel_df = df[rel_col].copy()

def nthHighest(row, n):
  ''' Takes a row and number n, and returns the row name with the nth highest value '''
  temp_dict = row.to_dict()
  sort = sorted(temp_dict.items(), key = lambda item: item[1], reverse = True)
  highest = sort[n-1][0]
  return highest

df['RELIGION'] = rel_df.apply(lambda row: nthHighest(row, 3), axis=1)

Aggregate Age

In [91]:
age_col = ['0-4 RATE', '5-9 RATE', '10-14 RATE', '15-19 RATE', '20-24 RATE',
       '25-29 RATE', '30-34 RATE', '35-39 RATE', '40-44 RATE', '45-49 RATE',
       '50-54 RATE', '55-59 RATE', '60-64 RATE', '65-69 RATE', '70-74 RATE',
       '75-79 RATE', '80-84 RATE', '85+ RATE']

df['AGE OLD'] = df['55-59 RATE'] + df['60-64 RATE'] + df['65-69 RATE'] +\
                df['70-74 RATE'] + df['80-84 RATE'] + df['85+ RATE']
df['AGE YOUNG'] = df['0-4 RATE'] + df['5-9 RATE'] + df['10-14 RATE'] +\
                  df['15-19 RATE'] + df['20-24 RATE']
print(df['AGE OLD'])

0       0.309524
1       0.184608
2       0.200555
3       0.196938
4       0.232126
          ...   
3135    0.227279
3136    0.223346
3137    0.326923
3138    0.234395
3139    0.211910
Name: AGE OLD, Length: 3140, dtype: float64


Aggregate Ethnicity

In [88]:
eth_f_col = ['TOT FEMALE RATE', 'WHITE FEMALE RATE', 'BLACK FEMALE RATE',
       'NATIVE AMERICAN FEMALE RATE', 'ASIAN FEMALE RATE',
       'HAWAIIAN PACIFIC FEMALE RATE','MULTI FEMALE RATE',]
eth_m_col = ['TOT MALE RATE', 'WHITE MALE RATE', 'BLACK MALE RATE',
             'NATIVE AMERICAN MALE RATE', 'ASIAN MALE RATE',
             'HAWAIIAN PACIFIC MALE RATE', 'MULTI MALE RATE',]
non_mf_ethnic_columns = ['BLACK RATE',
       'NATIVE AMERICAN RATE', 'HAWAIIAN PACIFIC RATE', 'MULTI RATE',]

eth_f_df = df[eth_f_col].copy()
eth_m_df = df[eth_m_col].copy()

df['ETHNIC MALE'] = eth_m_df.apply(lambda row: nthHighest(row, 2), axis=1)
df['ETHNIC FEMALE'] = eth_f_df.apply(lambda row: nthHighest(row, 2), axis=1)

Clean-up

In [89]:
del df['POVERTY UNDER 18 PCT']
del df['HOUSING UNITS']
del df['DENSITY PER SQUARE MILE OF LAND AREA - HOUSING UNITS']
del df['DENSITY PER SQUARE MILE OF LAND AREA - POPULATION']
del df['AGE TOTAL POP']
del df['AREA IN SQUARE MILES - LAND AREA']
del df['AREA IN SQUARE MILES - WATER AREA']
df = df.drop(
    ed_col + rel_col + eth_f_col + eth_m_col + non_mf_ethnic_columns,
    axis = 1,
)

4. Rename

In [90]:
#df.columns = df.columns.str.lower()
print(df.columns)

Index(['VOTES', 'UNEMPLOYMENT RATE', 'POP ESTIMATE', 'MED HHINC',
       'PER CAPITA INC', 'POVERTY ALL AGES PCT', 'DEEP POV ALL',
       'DEEP POV CHILDREN', 'POPULATION', 'AREA IN SQUARE MILES - TOTAL AREA',
       '0-4 RATE', '5-9 RATE', '10-14 RATE', '15-19 RATE', '20-24 RATE',
       '25-29 RATE', '30-34 RATE', '35-39 RATE', '40-44 RATE', '45-49 RATE',
       '50-54 RATE', '55-59 RATE', '60-64 RATE', '65-69 RATE', '70-74 RATE',
       '75-79 RATE', '80-84 RATE', '85+ RATE', 'WHITE RATE',
       'VOTER TURNOUT RATE', 'DEMOCRAT', 'name', 'STATE', 'EDUCATION',
       'RELIGION', 'ETHNIC MALE', 'ETHNIC FEMALE'],
      dtype='object')
